In [5]:
# importing the required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets as widgets

from IPython.display import display
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

In [6]:
# loading the data

df = pd.read_csv("C:\\Users\\Administrator\\Downloads\\fifa_players.csv")
df.head() # show top few rows

,name,full_name,birth_date,age,height_cm,weight_kgs,positions,nationality,overall_rating,potential,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,L. Messi,Lionel Andrés Messi Cuccittini,6/24/1987,31,170.18,72.1,"CF,RW,ST",Argentina,94,94,...,94,48,22,94,94,75,96,33,28,26
1,C. Eriksen,Christian Dannemann Eriksen,2/14/1992,27,154.94,76.2,"CAM,RM,CM",Denmark,88,89,...,89,46,56,84,91,67,88,59,57,22
2,P. Pogba,Paul Pogba,3/15/1993,25,190.50,83.9,"CM,CAM",France,88,91,...,82,78,64,82,88,82,87,63,67,67
3,L. Insigne,Lorenzo Insigne,6/4/1991,27,162.56,59.0,"LW,ST",Italy,88,88,...,84,34,26,83,87,61,83,51,24,22
4,K. Koulibaly,Kalidou Koulibaly,6/20/1991,27,187.96,88.9,CB,Senegal,88,91,...,15,87,88,24,49,33,80,91,88,87


In [7]:
df.columns # view column names and choose relevant stats

rating_cols = ['international_reputation(1-5)',
       'weak_foot(1-5)', 'skill_moves(1-5)', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'freekick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'composure', 'standing_tackle', 'sliding_tackle']

df[rating_cols].info() # check if all of these columns exist

# create weighted score
df['custom_score'] = (
    df['international_reputation(1-5)']* 0.03 +
    df['weak_foot(1-5)']* 0.02 +
    df['skill_moves(1-5)']* 0.02 +
    df['crossing']* 0.03 +
    df['finishing']* 0.04 +
    df['heading_accuracy']* 0.03 +
    df['short_passing']* 0.04 +
    df['volleys']* 0.03 +
    df['dribbling']* 0.05 +
    df['curve']* 0.02 +
    df['freekick_accuracy']* 0.02 +
    df['long_passing']* 0.03 +
    df['ball_control']* 0.04 +
    df['acceleration']* 0.03 +
    df['sprint_speed']* 0.03 +
    df['agility']* 0.03 +
    df['reactions']* 0.04 +
    df['balance']* 0.03 +
    df['shot_power']* 0.03 +
    df['jumping']* 0.02 +
    df['stamina']* 0.03 +
    df['strength']* 0.03 +
    df['long_shots']* 0.03 +
    df['aggression']* 0.02 +
    df['interceptions']* 0.02 +
    df['positioning']* 0.03 +
    df['vision']* 0.03 +
    df['penalties']* 0.02 +
    df['composure']* 0.03 +
    df['standing_tackle']* 0.02 +
    df['sliding_tackle']* 0.02
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17954 entries, 0 to 17953
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   international_reputation(1-5)  17954 non-null  int64
 1   weak_foot(1-5)                 17954 non-null  int64
 2   skill_moves(1-5)               17954 non-null  int64
 3   crossing                       17954 non-null  int64
 4   finishing                      17954 non-null  int64
 5   heading_accuracy               17954 non-null  int64
 6   short_passing                  17954 non-null  int64
 7   volleys                        17954 non-null  int64
 8   dribbling                      17954 non-null  int64
 9   curve                          17954 non-null  int64
 10  freekick_accuracy              17954 non-null  int64
 11  long_passing                   17954 non-null  int64
 12  ball_control                   17954 non-null  int64
 13  acceleration    

In [8]:
# Sort by 'best' players
top_players = df.sort_values(by='custom_score', ascending = False)

# Show 'top 10' players
top_players[['name', 'positions', 'custom_score']].head(10)

,name,positions,custom_score
0,L. Messi,"CF,RW,ST",68.27
17944,Cristiano Ronaldo,"ST,LW",68.02
13,A. Griezmann,"CF,ST",67.66
17924,G. Bale,"RW,LW,ST",67.47
17941,K. De Bruyne,"CAM,CM",67.40
17939,L. Modrić,CM,67.16
22,Marcelo,LB,67.04
2,P. Pogba,"CM,CAM",66.91
17938,L. Suárez,ST,66.77
17943,Neymar Jr,"LW,CAM",65.95


In [26]:
# Create interactive widgets
age_slider = widgets.IntSlider(value=30, min=16, max=45, step=1, description='Max Age : ')
unique_nationalities = sorted(df['nationality'].dropna().unique().tolist())
nationality_selector = widgets.SelectMultiple(
    options=unique_nationalities,
    value=('Brazil', 'Italy', 'Germany'),
    description='Nationalities : ',
    layout=widgets.Layout(height='150px', width='50%')
)

# Display widgets to the user
display(age_slider, nationality_selector)

# Get values from widgets
max_age = age_slider.value
allowed_nationalities = list(nationality_selector.value)

# Filter players by age and nationality
filtered_players = df[
    (df['age'] <= max_age) &
    (df['nationality'].isin(allowed_nationalities))
].copy()

# Select only necessary columns
filtered_players = filtered_players[['name', 'positions', 'custom_score', 'value_euro', 'age', 'nationality']]

# Map positions to group
def get_position_group(pos):
    posList = pos.split(',')
    if 'GK' in posList:
        return 'GK'
    elif any(x in posList for x in ['CB', 'RB', 'LB']):
        return 'DEF'
    elif any(x in posList for x in ['CM', 'CDM', 'CAM', 'LM', 'RM']):
        return 'MID'
    else:
        return 'FWD'

filtered_players['group'] = filtered_players['positions'].apply(get_position_group)

# Use filtered players in optimization
players = filtered_players

# Formation & Budget
formation = {'GK': 1, 'DEF': 4, 'MID': 3, 'FWD': 3}
budget = 100_000_000  # 100 million

# Linear Programming Setup
model = LpProblem("Optimal_Football_Team", LpMaximize)
player_vars = {i: LpVariable(name=f"player_{i}", cat='Binary') for i in players.index}

# Objective: Maximize custom_score
model += lpSum(player_vars[i] * players.loc[i, 'custom_score'] for i in players.index)

# Constraints
model += lpSum(player_vars[i] for i in players.index) == 11  # team size
model += lpSum(player_vars[i] * players.loc[i, 'value_euro'] for i in players.index) <= budget  # budget

# Formation constraints
for group, count in formation.items():
    model += lpSum(player_vars[i] for i in players.index if players.loc[i, 'group'] == group) == count

# Solve
model.solve()

# Output selected players
selected_players = players[[player_vars[i].varValue == 1.0 for i in players.index]]

print("\n*** OPTIMAL TEAM ***")
print(selected_players[['name', 'positions', 'group', 'custom_score', 'value_euro']])
print(f"\nTotal Team Score : {selected_players['custom_score'].sum():.2f}")
print(f"Total Team Value : €{selected_players['value_euro'].sum():,.0f}")

IntSlider(value=30, description='Max Age : ', max=45, min=16)

SelectMultiple(description='Nationalities : ', index=(18, 76, 57), layout=Layout(height='150px', width='50%'),…


*** OPTIMAL TEAM ***
                name positions group  custom_score  value_euro
301      A. Schürrle     RW,LW   FWD         59.64   9500000.0
538        Jefferson        LB   DEF         59.81   5000000.0
13949       K. Wolze     LB,LM   DEF         57.73   1400000.0
14067     M. Riemann        GK    GK         30.50   1200000.0
16677       Fabrício     ST,LW   FWD         57.65   7000000.0
16726     F. Viviani    CDM,CM   MID         59.36   6500000.0
17182       Maurício       CDM   MID         60.46   7000000.0
17464  Alex Teixeira    ST,CAM   MID         61.49  13000000.0
17474         Rafael    RB,RWB   DEF         60.74  10500000.0
17670      L. Stindl        CF   FWD         62.63  18000000.0
17766    A. Florenzi  RB,RW,LW   DEF         64.98  20500000.0

Total Team Score : 634.99
Total Team Value : €99,600,000


In [27]:
# Export optimal player configuration

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"optimal_team_{timestamp}.csv"
selected_players.to_csv(filename, index=False)

from IPython.display import FileLink
display(FileLink(filename))

c:\Users\Administrator\Desktop\Football Predictor\optimal_team_20250506_153919.csv